In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Local module
from nxr_classes import Organisations

In [4]:
nsr = pd.read_excel("nsr - alle organisasjoner.xlsx")

#### Nasjonalt skoleregister (NSR)
Udirs beskrivelse av registeret:   
"Data fra Nasjonalt skoleregister (NSR) viser enheter fra Virksomhets- og Foretaksregisteret (Vof) som tilhører grunnutdanningen. Skolene (bedriftene/virksomhetene) og deres eiere (foretak) skal være avstemt med VoF. NSR viser også Norske skoler i utlandet og inneholder noe tilleggsinformasjon utover det som fins i VoF."

Vi har hentet ut detaljert informasjon om samtlige 16 343 organisasjoner som er registrert i NSR.

I registeret er det registrert:
- Organisasjonsnummer
- Navn
- Skoleeier (kommune og fylke)
- Beliggenhet
- Kontaktinformasjon (epost, telefon og postadresse)
- Målform
- Organisasjonsform
- Skoleleder
- Elevtall
- Antall ansatte
- Skoletrinn grunnskolen (fra/til)
- Skoletrinn vgs (fra/til)
- Privatskolegodkjenning
- Næringskoder
- Skolekategori

Dette dokumentet går gjennom den mest sentrale informasjon i NSR. 

##### Aktive og nedlagte skoler

Først og fremst er det verdt å merke seg at NSR består av både aktive og nedlagte organsisasjoner. Tabellen under viser forskjellige status i registeret og hvor mange skoler som tilhører hver av dem. I NSR er det 8 332 aktive skoler, eller sagt på registerspråket: Skoler uten utgåtttype. 

In [17]:
nsr['Utgaattype_Navn'].value_counts()

Ingen utgåttype                        8334
Slettet                                6095
Utg pga eier mangler aktiv bedrift      709
Utg pga næringsendring                  696
Slettet for sammenslåing                416
Slettet for dublett                      51
Utg pga eier er Enkeltpersonforetak      46
Name: Utgaattype_Navn, dtype: int64

Det er også mulig å identifisere nedlagte organisasjoner med kolonnnen `ErAktiv`.

In [18]:
pd.crosstab(nsr.Utgaattype_Navn, nsr.ErAktiv)

ErAktiv,False,True
Utgaattype_Navn,,
Ingen utgåttype,2,8332
Slettet,6095,0
Slettet for dublett,51,0
Slettet for sammenslåing,416,0
Utg pga eier er Enkeltpersonforetak,46,0
Utg pga eier mangler aktiv bedrift,709,0
Utg pga næringsendring,696,0


##### Identifisere skolene
Det er ikke alle organisasjoner i NSR som er en skole. 9 540 av organisasjonene er registrert som skole, og samtlige av dem står oppført med organisasjonsformen `Bedrift`. Organisasjoner registrert i NSR som ikke er skoler er i ofte skoleeiere.  

In [19]:
pd.crosstab(nsr.Organisasjonsform_Navn, nsr.ErSkole)

ErSkole,False,True
Organisasjonsform_Navn,,
Aksjeselskap,2486,0
Annen juridisk person,34,0
Annet foretak iflg. særskilt lov,18,0
Ansvarlig selskap,50,0
Ansvarlig selskap med delt deltakeransvar,145,0
Bedrift,647,9542
Den norske kirke,2,0
Enkeltpersonforetak,456,0
Forening/lag/innretning,603,0


In [20]:
pd.crosstab(nsr.ErSkole, nsr.ErSkoleeier)

ErSkoleeier,False,True
ErSkole,,
False,2877,3928
True,9542,0


Fra nå av fokuserer vi bare på organisasjonene registrert som aktive skoler. Det er 5 567 organisasjoner som er registrert som aktive skoler.  

In [21]:
pd.crosstab(nsr.ErSkole, nsr.ErAktiv)

ErAktiv,False,True
ErSkole,,
False,4040,2765
True,3975,5567


In [22]:
skoler = nsr.query('ErAktiv == True & ErSkole == True')

##### Skolekategori

Skolene i NSR er registrert med forskjellige kategorier. Brukte kategorier, er: 

- Offentlig skole
- Privatskole
- Grunnskole
- Videregående skole
- Spesialskole
- Grunnopplæring

Som det fremgår av kategoriene kan én skole være registrert med flere skolekategorier. Når det er sagt er noen av kategoriene gjensidig utelukkende, slik som privat skole og offentligskole. Det er minimalt med overlapp mellom kategoriene grunnskole og videregående opplæring, men det er 42 skoler som er registrert i begge kategorier. 

Alle grunnskoler og videregående skoler er er naturlig definert med kategorien grunnopplæring. 

In [23]:
pd.crosstab(skoler.ErVideregaaendeSkole, skoler.ErGrunnskole)

ErGrunnskole,False,True
ErVideregaaendeSkole,,
False,2258,2761
True,506,42


In [24]:
pd.crosstab(skoler.ErGrunnopplaering, skoler.ErGrunnskole)

ErGrunnskole,False,True
ErGrunnopplaering,,
False,2204,0
True,560,2803


In [25]:
pd.crosstab(skoler.ErGrunnopplaering, skoler.ErVideregaaendeSkole)

ErVideregaaendeSkole,False,True
ErGrunnopplaering,,
False,2204,0
True,2815,548


Det er 2 204 skoler som hverken er grunnopplæring eller videregående skole. For å lære mer om disse skolene må vi se på skolekategoriene. En skole kan ha én eller flere skolekategorier. 

In [39]:
pd.crosstab(skoler.ErGrunnopplaering, skoler.ErVideregaaendeSkole)

ErVideregaaendeSkole,False,True
ErGrunnopplaering,,
False,2204,0
True,2815,548


1 247 av skolene har bare én skolekategori, og det er "Privat eiet".  

In [70]:
skoler_andre = skoler.query("ErGrunnopplaering == False & ErVideregaaendeSkole == False")

cols_skolekat = ["Orgnr"] + [col for col in skoler_andre if col.startswith("Skolekategorier") and col.endswith("_Navn")]

skolekategorier = skoler_andre[cols_skolekat]

skolekategorier = pd.melt(skolekategorier, id_vars ="Orgnr", value_vars=cols_skolekat[1:])
skolekategorier = skolekategorier[skolekategorier["value"].notna()]
skolekategorier.sort_values(by = "Orgnr")


skolekategorier["count"] = skolekategorier.groupby("Orgnr")['value'].transform(len)
skolekategorier_one = skolekategorier.query("count == 1")
skolekategorier_one["value"].value_counts()


Privat eiet    1247
Name: value, dtype: int64

865 av de resterende 957 skolene har kategorien "offentlig skole", og det er tydelig at dette i stor grad er kulturskoler og voksenopplæringssentre.  

In [74]:
skolekategorier_more = skolekategorier.query("count > 1")
skolekategorier_more["value"].value_counts()
#skolekategorier_more["Orgnr"].nunique()

Offentlig skole                           865
Kommunal                                  681
Kulturskole                               353
Voksenopplæringssenter                    286
Voksenopplæring på grunnskolens område    169
Statlig                                   129
Privat eiet                                92
Folkehøyskole                              88
Godkjent privatskole                       77
Fylkeskommunal                             54
Norsk skole i utlandet                      1
Interkommunal                               1
Name: value, dtype: int64